# `Classification Model`
* <b>File Name</b>                          : Reference_implementation of covid model in Vertex AI
* <b>Date of creation(dd-mm-yyyy)</b>       : 14-09-2022
* <b>Author Name/Dept</b>                   : AIShield
* <b>Organization</b>                       : BGSW
* <b>Description</b>                        : Source Code of reference implementation
* <b>Copyright</b>                          : Copyright 2023 Bosch Global Software Technologies Private Limited. All Rights Reserved.

###  `Metadata`
* Dataset: covid dataset
* Size of training set: 250
* Size of testing set : 25
* Number of class : 2
* Original Model: CNN

### `Outcomes`
* Accuracy of model: 88%

### `Install AIShield Package`

In [12]:
# !pip install aishield

### `Install the pre-requisite packages`

In [2]:
# !pip install numpy==1.24.1
# !pip install pandas==1.15
# !pip install requests==2.28.2
# !pip install google-cloud-aiplatform==1.21.0
# !pip install google-cloud-storage==2.7.0
# !pip install google-auth-oauthlib==0.4.6
# !pip install Pillow==9.4.0
# !pip install tensorflow==2.9.1
# !pip install opencv-python==4.7.0.68

Download the Google Cloud CLI Installer, and Install the same (can use gcloud commands which needed to upload download files on gcloud storage)
1. https://cloud.google.com/sdk/docs/install
2. open the terminal and do gcloud init --console-only (This will initialize the gcloud configuration and will autheticate the same)
3. refer the below link to installation of gcloud sdk (https://www.youtube.com/watch?v=PY8KnoCJpjo)

## 1.0 Importing neccessary libraries

In [3]:
'''
Description : Importing aishield libraries
'''

import aishield as ais

In [34]:
'''
Description : Importing libraries
'''

import os
import cv2
import time
import copy
import shutil
import json
import zipfile
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import storage
from google.cloud import aiplatform

## 2.0 Setting up gcloud instance

### Authenticate your Google Cloud account

#### 1. Vertex AI Workbench:

`Do nothing as you are already authenticated.`

#### 2. Local JupyterLab instance, uncomment and run:



In [5]:
'''
Description : to authenticate gcloud login
'''

! gcloud auth login

'\nDescription : to authenticate gcloud login\n'

#### 3. Colab, uncomment and run:

In [9]:
'''
Description : to authenticate gcloud login in colab
'''

# from google.colab import auth
# auth.authenticate_user()

'\nDescription : to authenticate gcloud login in colab\n'

###  `2.0.1 To access and run gcloud applications instances locally in jupyter-notebook`

#### 1. Vertex AI Workbench:

`Do nothing as you are already authenticated.`

#### 2. Local JupyterLab instance, uncomment and run:



In [1]:
'''
Description : to authenticate gcloud application login
'''

!gcloud auth application-default login

'\nDescription : to authenticate gcloud application login\n'

### Setting the application credentials as environmetal variable to use gcloud applications

1. when we run the command !gcloud auth application-default login
2. it will save the credentials to a file , can found as the output of running the above command.
3. **copy that path we got , and set as an environmental varibales to access and use the applications of gcloud** 

In [7]:
'''
Description : to set the credentials of gcloud application login to enviromnetal vaibale 
'''

## TODO copy the path we got as output of the above cell 
google_application_credentials_path = r"<PATH>"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = google_application_credentials_path

#### 3. Colab, uncomment and run:

In [10]:
'''
Description : to authenticate gcloud application login in colab
'''

# !pip install -U -q PyDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from oauth2client.client import GoogleCredentials

# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

'\nDescription : to authenticate gcloud application login in colab\n'

### `2.0.2 Set the Prpject ID , to jupyter-notebook`

######  below project_id and region we will use while creating any job instance or bucket 


In [8]:
'''
Description : Set the Project id 
'''

##TODO copy and paste project_id in plcae of <PROJECT_ID>
PROJECT_ID = "<PROJECT_ID>"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [9]:
'''
Description : Set the region and bucket name
'''

##TODO copy and paste region , bucket_name  in plcae of <REGION_NAME> and <BUCKET_NAME>

REGION = "<REGION_NAME>" # @param {type:"string"}
BUCKET_NAME = "<BUCKET_NAME>" # @param {type:"string"}
BUCKET_URI = "gs://"+"<BUCKET_NAME>" # @param {type:"string"}

### uncomment  the below cell , if you want to create a bucket

In [10]:
#TODO copy and paste the necessary info in {REGION}, {PROJECT_ID} AND {BUCKE_URI}

# !gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

In [1]:
'''
Description : this will print the list of buckets present respective to the given PROJECT_ID
'''

storage_client = storage.Client(project=PROJECT_ID)
buckets = storage_client.list_buckets()
print("Buckets:")
for bucket in buckets:
    print(bucket.name)

'\nDescription : this will print the list of buckets present respective to the given PROJECT_ID\n'

## 3.0 Uploading the dataset

##### uncomment it if you want to upload the dataset in to gcloud bucket 

In [2]:
'''
Description : Upload the dataset from local directory to gcloud bucket 
'''

data_path ="Training_Dataset/data/"

#TODO copy and paste the BUCKET_URI in place of {BUCKET_URI}
!gsutil cp -r "Training_Dataset/data/" "{BUCKET_URI}"

'\nDescription : Upload the dataset from local directory to gcloud bucket \n'

## 4.0 Initialize Vertex AI SDK for Python

In [13]:
'''
Description : to initialize the vertex ai platform sdk
'''

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## 5.0 Train and Download the Model

### `5.0.0 Preparing a traning script`

In [15]:
'''
Description : to create a training script
'''

# %%writefile train.py

python_code ='''
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import datasets, layers, optimizers , Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, Flatten
from tensorflow.keras.regularizers import l1, l2
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2
import os
import sys
import random
from random import shuffle
import json
import time
import argparse
from google.cloud import storage

img_row,img_col,channel=(32,32,1)
num_classes=2
input_shape=(img_row,img_col,channel)

label_int_str_map = {0:'no_findings',1:'covid'}

def prepare_data(file_path,classes=2):
    """
    Description: load data and perform preprocessing  
    Args:
        file_path:path where data is present
        classes: number of class where to load data - 
    Returns: numpy format data and its label
    """
    data = []
    labels = []
    download_path = "gsutil cp -r "+file_path+" ."
    print("Downloading files from gcloud bucket")
    os.system(download_path)
    print("Downloaded data from gcloud bucket")
    path_file = file_path.split("/")[-1]
    for i in range(classes):
        path = os.path.join(path_file,str(i))
        images = os.listdir(path)
        for a in images:
            try:
                img = cv2.imread("./"+os.path.join(path,str(a)), 0)
                img = cv2.resize(img, (input_shape[1], input_shape[0])) 
                img=np.expand_dims(img,axis=-1)
                data.append(img)
                labels.append(i)
            except Exception as e:
                print(e)
    return np.array(data),np.array(labels) 

def modelarch():
    a=0.07
    model = Sequential()
    model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',strides=(1,1), activation='relu',input_shape=(32,32,1), 
                     kernel_regularizer=l2(a), bias_regularizer=l2(a)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',strides=(1,1), activation='relu',kernel_regularizer=l2(a), bias_regularizer=l2(a)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',strides=(1,1), activation='relu',kernel_regularizer=l2(a), bias_regularizer=l2(a)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',strides=(1,1), activation='relu',
                     kernel_regularizer=l2(a), bias_regularizer=l2(a)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dropout(0.3))

    model.add(Dense(10,activation='selu', kernel_regularizer=l2(a), bias_regularizer=l2(a)))
    model.add(Dropout(0.3))

    model.add(Dense(num_classes,activation = 'softmax'))

    return model

def train(train_data_folder,BUCKET_NAME, PROJECT_ID):
    print("Preparing data")
    X,y = prepare_data(train_data_folder,classes=num_classes)
    print("Splitting the dataset")
    X_train,X_test , y_train , y_test= train_test_split(X,y,stratify = y,test_size = 0.2,shuffle=True,random_state = 42)
    print("Splitting the dataset")
    X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,stratify = y_train,test_size = 0.1,shuffle=True,random_state = 42)
    X_train = X_train.reshape(-1,*input_shape)/255.0
    X_val =  X_val.reshape(-1,*input_shape)/255.0
    X_test =  X_test.reshape(-1,*input_shape)/255.0
    y_train=keras.utils.to_categorical(y_train,num_classes)
    y_val=keras.utils.to_categorical(y_val,num_classes)
    y_test=keras.utils.to_categorical(y_test,num_classes)

    model = modelarch()
    
    #Compile model
    model.compile(optimizer=optimizers.Adam(), loss = 'categorical_crossentropy', metrics=['accuracy']) 
    
    # Checkpoint
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='outputs/model.h5',monitor='val_loss',verbose=1,save_best_only=True,mode='auto')
    # Early stopper
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=10,mode='min')

    callbacks = [early_stop, checkpoint]

    history = model.fit(X_train, y_train, validation_data = (X_val, y_val),epochs = 100, batch_size=16, verbose = 1 , callbacks = callbacks)
    
    print("Finished Training")
    
    print("saving the model file")
    
    model.save('covid_model.h5')
    
    upload_blob(BUCKET_NAME,'covid_model.h5', 'models/covidexp/covid_model.h5', PROJECT_ID)

def upload_blob(bucket_name, source_file_name, destination_blob_name, PROJECT_ID):
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
    

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--train-data-folder', type=str, dest='train_data_folder',
    help='data folder mounting point')
    parser.add_argument('--project-id', type=str, dest='PROJECT_ID')
    parser.add_argument('--bucket-name', type=str, dest='BUCKET_NAME')
    args = parser.parse_args()
    train_data_folder = args.train_data_folder
    BUCKET_NAME = args.BUCKET_NAME
    PROJECT_ID = args.PROJECT_ID
    print("===== DATA =====")
    print('Train Data folder:', train_data_folder)
    train(train_data_folder=train_data_folder, BUCKET_NAME = BUCKET_NAME, PROJECT_ID = PROJECT_ID)'''



with open("train.py", "w") as file:
    # Writing data to a file
    file.writelines(python_code)

UsageError: Line magic function `%%writefile` not found.


### `5.0.1 Preparing a Vertex AI Custom containers to run training`

#### `Preparing a Docker file`

In [4]:
'''
Description : create a dockerfile
'''

# %%writefile Dockerfile

docker_code = '''
FROM tensorflow/tensorflow:2.9.1-gpu
WORKDIR /root

RUN apt-get install python3
RUN apt-get install python3-pip
RUN apt update && apt install -y libsm6 libxext6
RUN apt-get install -y libxrender-dev


RUN apt-get update && \
    apt-get install -y curl gnupg && \
    echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && \
    curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key --keyring /usr/share/keyrings/cloud.google.gpg  add - && \
    apt-get update -y && \
    apt-get install google-cloud-sdk -y
    
RUN pip3 install opencv-python numpy==1.22 google-cloud-storage==1.35.0 scikit-learn==1.0.2 pandas==1.1.5 protobuf==3.20.*

COPY train.py ./train.py

ENTRYPOINT ["python3", "train.py"]'''

with open('Dockerfile', "w") as file:
    # Writing data to a file
    file.writelines(docker_code)

'\nDescription : create a dockerfile\n'

#### `Preparing a cloud_build file`

#### before writing the cloud_build file , replace the < PROJECT_ID > and < BUCKET_NAME >

for eg - args: ['build', '-t', 'gcr.io/xyz-sandbox/xyz', '.']

In [5]:
'''
Description : Create a cloub_build file 
'''

# %%writefile cloudfile2.yaml

#TODO copy and paste the PROJECT_ID, BUCKET_NAME in place of <PROJECT_ID> and <BUCKET_NAME>

cloud_file_code ='''
steps:
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/<PROJECT_ID>/<BUCKET_NAME>', '.']
    
- name: 'gcr.io/cloud-builders/docker'
  args: ['push', 'gcr.io/<PROJECT_ID>/<BUCKET_NAME>']
images:
- gcr.io/<PROJECT_ID>/<BUCKET_NAME>'''


with open('cloudfile2.yaml', "w") as file:
    # Writing data to a file
    file.writelines(cloud_file_code)

'\nDescription : Create a cloub_build file \n'

#### `Run the cloud_build to create a custom image`

In [6]:
'''
Description : to build the custom container image 
'''
#TODO copy and paste the REGION in place of REGION


!gcloud builds submit --config cloudfile2.yaml --region=REGION

'\nDescription : to build the custom container image \n'

### `5.0.2 Creating a Job id to run the traning script`

In [25]:
'''
Description : Create a JOB name to run the training script
'''
JOB_NAME = "covid_job_ais_final_1"

DATA_DIR = BUCKET_URI+"/data"
# CMDARGS = ['--train-data-folder', DATA_DIR]
CMDARGS = [
    "--train-data-folder="+DATA_DIR,
    "--project-id="+PROJECT_ID,
    "--bucket-name="+BUCKET_NAME
]

In [26]:
'''
Description : Copy the Custom image name with tag created when we ran the Cloud_build file
'''

#TODO copy and paste the PROJECT_ID,BUCKET_NAME  in place of <PROJECT_ID> and <BUCKET_NAME>

CUSTOM_CONTAINER_IMAGE_URI="gcr.io/<PROJECT_ID>/<BUCKET_NAME>:latest"

In [7]:
'''
Description : Specify the worker pool configuration , and provide image_uri path and any arguments need to pass
'''
#this below worker_pool_specs uses tensorflow2.9.1-gpu (machine_type and acccelarator_type)

worker_pool_specs = [
        {
            "machine_spec": {
                "machine_type": "n1-standard-8",
                "accelerator_type": "NVIDIA_TESLA_T4",
                "accelerator_count": 1,
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": CUSTOM_CONTAINER_IMAGE_URI,
                "args" :CMDARGS
            },
        }
    ]
print(worker_pool_specs)

'\nDescription : Specify the worker pool configuration , and provide image_uri path and any arguments need to pass\n'

In [28]:
'''
Description : Creating a custom traning job using vertex ai sdk
'''

job = aiplatform.CustomJob(
    display_name=f"{JOB_NAME}",
    worker_pool_specs=worker_pool_specs,    
)


In [8]:
'''
Description : this cell will run the job id for the custom training 
'''
job.run(
    sync=False,
)

'\nDescription : this cell will run the job id for the custom training \n'

In [9]:
'''
Description : Can found the configuration of job which has run 
'''
job.gca_resource

'\nDescription : Can found the configuration of job which has run \n'

### `5.0.3 Download the model saved inside bucket of aishield and save to locally`

In [10]:
'''
Description : to download the model from gcloud bucket to locally
'''

#TODO copy and paste the BUCKET_URI  in place of BUCKET_URI

!gsutil cp -r BUCKET_URI+"/models/covidexp/" .

'\nDescription : to download the model from gcloud bucket to locally\n'

In [35]:
'''
Description : to load the training model
'''

imported_model = tf.keras.models.load_model('covidexp/covid_model.h5')

In [36]:
imported_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 256)       2560      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 256)      0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 16, 16, 256)      1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 16, 16, 256)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0

### 6.0 AIShield pypi Call

### `6.0.1 Prepare Data , Model and Label zip`

In [38]:
def make_directory(directory):
    """
    create directory

    Parameters
    ----------
    directorys : list containing the directorys path to create 
    Returns
    -------
    None.

    """
    for d in directory:
        if os.path.isdir(d):
            print("directory {} already exist".format(d))
        if os.path.isdir(d)==False:
            os.mkdir(path=d)
            print("directory {} created successfully".format(d))

In [39]:
def delete_directory(directorys):
    """
    delete directory 

    Parameters
    ----------
    directorys : list containing the directorys to deleate along with all the files

    Returns
    -------
    None.

    """
    if len(directorys)>=1:
        for d in directorys:
            if os.path.isdir(d):
                try:
                    if os.path.isfile(d):
                        os.remove(path=d)
                    else:
                        shutil.rmtree(path=d)
                        print("Removed: {}".format(d))
                except:
                    print("Failed to removed: {}".format(d))
            else:
                print("Failed to removed: {}".format(d))
                

In [40]:
def make_archive(base_name,root_dir,zip_format='zip'):
    """
    created zip for given folder

    Parameters
    ----------
    base_name : name of zip file
    root_dir : directory to archive/zip
    zip_format : zip or tar 
        DESCRIPTION. The default is 'zip'.

    Returns
    -------
    None.

    """
    shutil.make_archive(base_name=base_name, format=zip_format, root_dir=root_dir)
    

In [41]:
def prepare_data(file_path,classes=2):
    """
    Description: load data and perform preprocessing  
    Args:
        file_path:path where data is present
        classes: number of class where to load data - 
    Returns: numpy format data and its label
    """
    data = []
    labels = []
    for i in range(classes):
        path = os.path.join(file_path,str(i))
        images = os.listdir(path)
        for a in images:
            try:
                img = cv2.imread(os.path.join(path,str(a)), 0)
                img = cv2.resize(img, (input_shape[1], input_shape[0]))
                img=np.expand_dims(img,axis=-1)
                data.append(img)
                labels.append(i)
            except Exception as e:
                print(e)
    return np.array(data),np.array(labels)

In [11]:
'''
Description : Create data, model and label folder
'''
data_path=os.path.join(os.getcwd(),"data")
model_path=os.path.join(os.getcwd(),"model")
label_path=os.path.join(os.getcwd(),"label")
zip_path=os.path.join(os.getcwd(),"zip")
pyc_model_path=os.path.join(os.getcwd(),"pyc_model")
report_path = os.path.join(os.getcwd(), "reports")
#deleting folder
delete_directory(directorys=[data_path,model_path,label_path,zip_path,pyc_model_path,report_path])

#creating folder
make_directory([data_path,model_path,label_path,zip_path,pyc_model_path,report_path])

'\nDescription : Create data, model and label folder\n'

In [43]:
input_shape = (32,32,1)
num_classes = 2

In [46]:
training_images_path = '../Training_Dataset/data/'

In [47]:
data , label = prepare_data(training_images_path,classes=2)

In [48]:
data.shape , label.shape

((250, 32, 32, 1), (250,))

In [49]:
"""
Description: Save data and label
"""
labels=pd.DataFrame()
img_name = []
img_label = [] 
  
for i in range(data.shape[0]):
    cv2.imwrite(os.path.join(data_path,str(i)+".jpg"),data[i]) # don't use plt.imread otheriwse while loading the saved images , and passing to model there is accuarcy drop 
    img_name.append(str(i)+".jpg")
    img_label.append(label[i])
labels['image'] = img_name
labels["label"] = np.array(img_label)

#write orig_label dataframe
labels.to_csv(os.path.join(label_path,"label.csv"),index=False)

In [50]:
"""
Description: Zip data
"""
make_archive(base_name=os.path.join(zip_path,"data"),root_dir=data_path,zip_format='zip')


In [51]:
"""
Description: Zip label
"""
make_archive(base_name=os.path.join(zip_path,"label"),root_dir=label_path,zip_format='zip')

In [52]:
"""
Description: Zip model
"""
shutil.copy('covidexp/covid_model.h5',model_path)
make_archive(base_name=os.path.join(zip_path,"model"),root_dir=model_path,zip_format='zip')

### `6.0.2 AIShield API Call`

In [61]:
"""
Description: AIShield API URL and subscription key
"""
url = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
headers={'Cache-Control': 'no-cache',
'Org-Id': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
'x-api-key': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
}


####  Perform Vulnerability Analysis through VulConfig Function


In [62]:
"""
Description: Files path
"""
data_path=os.path.join(zip_path,'data.zip') #full path of data zip
label_path=os.path.join(zip_path,'label.zip') #full path of label zip
model_path=os.path.join(zip_path,'model.zip') #full path of model zip

In [64]:
"""
Description: Initialize the AIShield API
"""
client = ais.AIShieldApi(api_url=url, api_key=api_key, org_id=org_id)

In [66]:
"""
Description: Define the task and analysis type
"""
task_type = ais.get_type("task", "image_classification")
analysis_type = ais.get_type("analysis", "extraction")

### Model registration

In [67]:
"""
Description: Perform model registration and upload the input artifacts
"""
status, job_details = client.register_model(task_type=task_type, analysis_type=analysis_type)
model_id = job_details.model_id
# print('Model id: {} \nInput artifacts will be uploaded as:\n data_upload_uri: {}\n label_upload_uri: {}'
#       '\n model_upload_uri: {}'.format(model_id, job_details.data_upload_uri, job_details.label_upload_uri,
#                                        job_details.model_upload_uri))

upload_status = client.upload_input_artifacts(job_details=job_details,
                                              data_path=data_path,
                                              label_path=label_path,
                                              model_path=model_path, )
print('Upload status: {}'.format(', '.join(upload_status)))

Upload status: data file upload successful, label file upload successful, model file upload successful


### Model Analysis

In [68]:
"""
Description: Specify the appropriate configs required for vulnerability analysis
"""


vuln_config = ais.VulnConfig(task_type=task_type,
                             analysis_type=analysis_type,
                             defense_generate=True)

vuln_config.input_dimensions = input_shape  # input dimension for mnist digit classification
vuln_config.number_of_classes = num_classes  # number of classes for mnist digit classification
vuln_config.attack_type = "greybox"  # greybox or blackbox depending upon the availability of information about Model, Data and Parameters
vuln_config.number_of_attack_queries = 10000  # Number of attack queries to be generated for testing model vulnerability 
vuln_config.encryption_strategy = 0  # value 0 (or) 1, if model is unencrypted or encrypted(pyc) respectively
print('IC-Extraction parameters are: \n {} '.format(vuln_config.get_all_params()))

IC-Extraction parameters are: 
 {'attack_type': 'greybox', 'defense_bestonly': 'no', 'encryption_strategy': 0, 'input_dimensions': '(32, 32, 1)', 'model_api_details': '', 'model_framework': 'tensorflow', 'normalize_data': 'yes', 'number_of_attack_queries': 10000, 'number_of_classes': 2, 'use_model_api': 'no', 'vulnerability_threshold': 0} 


In [12]:

"""
Description: Run vulnerability analysis
"""
my_status, job_details = client.vuln_analysis(model_id=model_id, vuln_config=vuln_config)
my_job_id = job_details.job_id
print('status: {} \nJob_id: {} '.format(my_status, my_job_id))

status: success 
Job_id: XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX 


In [13]:
"""
Description: Monitor progress for given Job ID using the Link below
"""
print('Click on the URL to view Vulnerability Dashboard (GUI): {}'.format(job_details.job_monitor_uri))

'\nDescription: Monitor progress for given Job ID using the Link below\n'

In [1]:
"""
Description: Fetch Job status using Job ID
"""
my_status = client.job_status (job_id = my_job_id)
print('job status ', my_status)

'\nDescription: Fetch Job status using Job ID\n'

### 7.0 Upload the Reports and Artifacts to gcloud bucket

In [14]:
"""
Description: Creating a directory to save the defense artifacts
"""
OUTPUT_PATH = os.path.join(os.getcwd(),"Output_Artifacts")
make_directory([OUTPUT_PATH])

'\nDescription: Creating a directory to save the defense artifacts\n'

In [16]:
"""
Description: Download the Attack Samples
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "attack_samples"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

'\nDescription: Download the Attack Samples\n'

In [17]:
"""
Description: Download the Vulnerability Report
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "vulnerability"),
                                 file_format=ais.get_type("file_format", "pdf"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

'\nDescription: Download the Vulnerability Report\n'

In [18]:
"""
Description: Download the defense Report
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "defense"),
                                 file_format=ais.get_type("file_format", "pdf"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

'\nDescription: Download the Vulnerability Report\n'

In [19]:
"""
Description: Download the defense_artifact Report
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "defense_artifact"),
                                 file_format=ais.get_type("file_format", "pdf"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

'\nDescription: Download the Vulnerability Report\n'

In [80]:
def zip_extractor(file, extract_path=None, delete_zip=False):
    """
    extract zip file to the given path

    Parameters
    ----------
    file : path of zip file
    extract_path : path to extract zip file, default considered parent directory
    delete_zip: True, delete zip file after unzipping it

    Returns
    -------
    None.
    """
    if extract_path is None:
        extract_path = os.path.dirname(file)
    print("Extracting : {}".format(file))
    zf = zipfile.ZipFile(file=file, mode='r')
    zf.extractall(extract_path)
    zf.close()
    if delete_zip:
        os.remove(file)
        print("{} removed successfully.".format(file))


In [81]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

In [83]:
print(os.listdir(OUTPUT_PATH))

['attack_samples_20230327_1831.zip', 'defense_20230327_1943.pdf', 'defense_artifact_20230327_1943.zip', 'vulnerability_20230327_1830.pdf']


In [229]:
"""
Description: download generated artifact
"""
upload_blob(BUCKET_NAME,'reports/Attack_samples.zip', 'report_artifacts/Attack_samples.zip')

File reports/Attack_samples.zip uploaded to report_artifacts/Attack_samples.zip.


In [226]:
"""
Description: upload defense_artifcat to gcloud
"""
upload_blob(BUCKET_NAME,'reports/Defense_artifact.zip', 'report_artifacts/Defense_artifact.zip')

File reports/Defense_artifact.zip uploaded to report_artifacts/Defense_artifact.zip.


In [227]:
"""
Description: upload vulnerabilirt report to gcloud
"""
upload_blob(BUCKET_NAME,'reports/Vulnerability.zip', 'report_artifacts/Vulnerability.zip')

File reports/Vulnerability.zip uploaded to report_artifacts/Vulnerability.zip.


In [228]:
"""
Description: upload defense report to gcloud
"""
upload_blob(BUCKET_NAME,'reports/Defense.zip', 'report_artifacts/Defense.zip')

File reports/Defense.zip uploaded to report_artifacts/Defense.zip.


### 8.0 Deploy the models

In [11]:
"""
Description: Extracting defense artifact
"""
zip_extractor(file=os.path.join(report_path, 'Defense_artifact.zip'))

'\nDescription: Extracting defense artifact\n'

In [237]:
defense_model = tf.keras.models.load_model("reports/defense_model.h5")

In [238]:
defense_model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 original_img (InputLayer)   [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 30, 30, 16)        160       
                                                                 
 conv2d_19 (Conv2D)          (None, 28, 28, 32)        4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 9, 9, 32)         0         
 2D)                                                             
                                                                 
 dense_16 (Dense)            (None, 9, 9, 64)          2112      
                                                                 
 global_max_pooling2d_8 (Glo  (None, 64)               0         
 balMaxPooling2D)                                          

In [239]:
"""
Description: save defense model in .pb format
"""

defense_model.save('tf_defense')

INFO:tensorflow:Assets written to: tf_defense\assets


INFO:tensorflow:Assets written to: tf_defense\assets


In [242]:
"""
Description: upload the pb format defense model to gcloud bucket
"""
#TODO copy and paste the BUCKET_NAME  in place of <BUCKET_NAME>

!gsutil cp -r "tf_defense/" "gs://<BUCKET_NAME>/tf_defense"

Copying file://tf_defense\keras_metadata.pb [Content-Type=application/octet-stream]...
/ [0 files][    0.0 B/ 15.2 KiB]                                                
/ [1 files][ 15.2 KiB/ 15.2 KiB]                                                
-
Copying file://tf_defense\saved_model.pb [Content-Type=application/octet-stream]...
- [1 files][ 15.2 KiB/152.7 KiB]                                                
- [2 files][152.7 KiB/152.7 KiB]                                                
\
Copying file://tf_defense\variables\variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
\ [2 files][152.7 KiB/243.5 KiB]                                                
\ [3 files][243.5 KiB/243.5 KiB]                                                
|
Copying file://tf_defense\variables\variables.index [Content-Type=application/octet-stream]...
| [3 files][243.5 KiB/245.7 KiB]                                                
| [4 files][245.7 KiB/245.7 KiB]                    

In [20]:
"""
Description: Do a model registry to deloy the model
"""
#TODO copy and paste the BUCKET_NAME  in place of <BUCKET_NAME>
# here we have used the prebuilt container present in vertext ai for deployment

my_model = aiplatform.Model.upload(display_name='covid-defense-model',
                                  artifact_uri='gs://<BUCKET_NAME>/tf_defense',
                                  serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest',
                                  project=PROJECT_ID)

'\nDescription: Do a model registry to deloy the model\n'

In [21]:
"""
Description: create an endoint to deploy the model 
"""

endpoint = my_model.deploy(
     deployed_model_display_name='my-defense-endpoint',
     traffic_split={"0": 100},
     machine_type="n1-standard-4",
     accelerator_count=0,
     min_replica_count=1,
     max_replica_count=1,
   )

'\nDescription: create an endoint to deploy the model \n'

In [22]:
"""
Description: endpoint.display_name
"""

endpoint.display_name

'\nDescription: endpoint.display_name\n'

In [23]:
"""
Description: endpoint.gca_resource
"""

endpoint.gca_resource

'\nDescription: endpoint.gca_resource\n'

In [24]:
"""
Description: reload the endpoint once deployed
"""

#TODO copy and paste the endpoint_name, we got from the above cells output

endpoint_name = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
endpoint = aiplatform.Endpoint(
    endpoint_name=endpoint_name)

'\nDescription: reload the endpoint once deployed\n'

In [264]:
x_test = data[1:10].tolist()

In [266]:
"""
Description: access endpoint for doing prediction
"""

print(endpoint.predict(instances=x_test).predictions)

[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]]


In [25]:
"""
Description: undeploy the endpoint once done the prediction
"""

#TODO copy and paste the endpoint_id, we got from the above cells output (id)

endpoint_id = "XXXXXXXXXXXXXXXXXXXXXXXXX"
endpoint.undeploy(endpoint_id)

'\nDescription: undeploy the endpoint once done the prediction\n'